<a href="https://colab.research.google.com/github/Riky2014/Tesi/blob/main/1d_hemo_solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!apt-get install software-properties-common
!add-apt-repository -y ppa:fenics-packages/fenics
!apt-get update -qq
!apt install fenics

In [2]:
from fenics import *
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Data
A0 = 4e-4
q0 = 0
k_r = 2.416
rho = 1.05
alpha = 1

T = 1
x_left = 0
x_right = 1.0

# Discretization parameter
dt = 1e-5
h = 1e-2
num_steps = T / dt
N = int((x_right - x_left) / h)

#  Create a mesh on the interval [0, 1].
mesh = IntervalMesh(N, x_left, x_right)
x = MeshCoordinates(mesh)

# Define the function space
P1 = FiniteElement('P', mesh.ufl_cell(), 1)
element = MixedElement([P1, P1])
V = FunctionSpace(mesh, element)

In [4]:
# Exact solution
L = 1
A_tilde = 4e-4
a_tilde = 4e-5
q_tilde = 0.0
A_exact = Expression('A_tilde + a_tilde * sin(2 * pi / L * x[0]) * cos(2 * pi / T * t)', A_tilde = A_tilde, a_tilde = a_tilde, L = L, T = T, degree = 2, t = 0)
q_exact = Expression('q_tilde - (a_tilde * L / T) * cos(2 * pi / L * x[0]) * sin(2 * pi / T * t)', q_tilde = q_tilde, a_tilde = a_tilde, L = L, T = T,degree = 2, t = 0)

In [19]:
def U(A, q):
  return as_vector([A, q])

def H(A, q):
  return as_tensor([[0, 1], [1190476.19048 * A ** 0.5 - (q / A) ** 2, 2 * q / A]])

def F(A, q):
  return as_vector([q, 793650.79365 * A ** 1.5 - 6.34921 + q ** 2 / A])

def B(A, q):
  return as_vector([0, k_r * q / A])

def S(A, q):
  return as_vector([0, k_r * q / A])

def dS_dU(A, q):
  return as_tensor([[0, 0], [- k_r * q / A ** 2, k_r / A]])

In [102]:
def c_alpha(A, q, alpha = 1):
  return (1190476.19048 * A ** 0.5 + (q / A) ** 2 * alpha * (alpha - 1)) ** 0.5

def l1(A, q, alpha = 1):
  return as_vector([c_alpha(A, q, alpha) - alpha * q / A, 1])

def l2(A, q, alpha = 1):
  return as_vector([- c_alpha(A, q, alpha) - alpha * q / A, 1])

def CC(A, q):
  return U(A, q) - dt * dot(H(A, q), U(A, q).dx(0)) - dt * B(A, q)

In [ ]:
# Boundary condition
inflow = 'near(x[0], 0)'
outflow = 'near(x[0], 1)'

# Dirichlet bc
bc_A_inflow = DirichletBC(V.sub(0), Constant(A0), inflow)
# I should impose compatibility condition
bc_q_inflow = DirichletBC(V.sub(1), , inflow)

# Non reflecting bc
bc_A_outflow = DirichletBC(V.sub(0), , outflow)
# I should impose compatibility condition
bc_q_outflow = DirichletBC(V.sub(1), , outflow)

In [111]:
# Define initial guess
u_old = interpolate(Expression(('A0', 'q0'), degree = 2, A0 = A0, q0 = q0), V)
A_old, q_old = u_old.split()

In [ ]:
#U(A_old, q_old)[0].compute_vertex_values(mesh)[0]
#l2(A_old, q_old)[0](mesh.coordinates()[0])
#CC(A_old, q_old)[0](mesh.coordinates()[0])

In [15]:
# Define trial functions and test functions
A, q = TrialFunctions(V)
v, z = TestFunctions(V)

# Define the source term
f = Expression(('0','5.98399 * pow((0.1 * sin(6.28319*x[0]) * cos(6.28319*t) + 1), 0.5) * cos(6.28319*t) * cos(6.28319*x[0]) - 0.00025 * cos(6.28319*t) * cos(6.28319*x[0]) - 0.0001 * sin(6.28319*t) * cos(6.28319*x[0]) / (4.0e-5 * sin(6.28319*x[0]) * cos(6.28319*t) + 0.0004)'), degree = 2, t = 0)
df_dt = Expression(('0', '-1.87993 * sin(6.28319*t) * sin(6.28319*x[0]) * cos(6.28319*t) * cos(6.28319*x[0]) / pow((0.1 * sin(6.28319*x[0]) * cos(6.28319*t) + 1), 0.5) - 37.5985 * pow((0.1 * sin(6.28319*x[0]) * cos(6.28319*t) + 1), 0.5) * sin(6.28319*t) * cos(6.28319*x[0]) + 0.00157 * sin(6.28319*t) * cos(6.28319*x[0]) - 0.15752 * pow(sin(6.28319*t), 2) * sin(6.28319*x[0])* cos(6.28319*x[0]) / pow((0.1 * sin(6.28319*x[0]) * cos(6.28319*t) + 1), 2) - 0.00063 * cos(6.28319*t) * cos(6.28319*x[0]) / (4.0e-5 * sin(6.28319*x[0]) * cos(6.28319*t) + 0.0004)'), degree = 2, t = 0)

f_n = interpolate(f, V)
df_dt_n = interpolate(df_dt, V)

# Define the variational problem
a = inner(A, v) * dx + inner(q, z) * dx

L = (
      A_old * v * dx
    + q_old * z * dx
    + dt * ((F(A_old, q_old) - dt / 2 * dot(H(A_old, q_old), S(A_old, q_old)))).dx(0)[0] * v * dx
    + dt * ((F(A_old, q_old) - dt / 2 * dot(H(A_old, q_old), S(A_old, q_old)))).dx(0)[1] * z * dx
    + dt ** 2 / 2 * (dot(dS_dU(A_old, q_old), F(A_old, q_old).dx(0)) + (dot(H(A_old, q_old), F(A_old, q_old).dx(0))).dx(0))[0] * v * dx
    + dt ** 2 / 2 * (dot(dS_dU(A_old, q_old), F(A_old, q_old).dx(0)) + (dot(H(A_old, q_old), F(A_old, q_old).dx(0))).dx(0))[1] * z * dx
    - dt * (S(A_old, q_old) - dt / 2 * dot(dS_dU(A_old, q_old), S(A_old, q_old)))[0] * v * dx
    - dt * (S(A_old, q_old) - dt / 2 * dot(dS_dU(A_old, q_old), S(A_old, q_old)))[1] * z * dx
    + dt * f[0] * v * dx
    + dt * f[1] * z * dx
    + dt ** 2 / 2 * (- (dot(H(A_old, q_old), f_n)).dx(0) - dot(dS_dU(A_old, q_old), f_n) + df_dt_n)[0] * v * dx
    + dt ** 2 / 2 * (- (dot(H(A_old, q_old), f_n)).dx(0) - dot(dS_dU(A_old, q_old), f_n) + df_dt_n)[1] * z * dx
)

In [ ]:
# Time stepping
u = Function(V)
t = 0
i = 1
for n in range(int(num_steps) + 1):

  i +=1

  # Update time step
  t += dt
  f.t = t
  df_dt.t = t
  A_exact.t = t
  q_exact.t = t

  # Solve the problem
  solve(a == L, u)
  A, q = u.split(deepcopy = True)

  # Compute errors
  if (i % 100 == 0):
    print(f'Interation {i} / {int(num_steps) + 1}')
    A_e = interpolate(A_exact, V.sub(0).collapse())
    q_e = interpolate(q_exact, V.sub(1).collapse())
    error = np.array([errornorm(A, A_e, 'L2') / norm(A_e, 'L2'), errornorm(q, q_e, 'L2') / norm(q_e, 'L2')])
    print(f't = %.4f: error = {error}' % (t))
    print()

  # Update previous solution
  A_old.assign(A)
  q_old.assign(q)

In [10]:
A, q = u.split()
A.compute_vertex_values(mesh), q.compute_vertex_values(mesh)

(array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]),
 array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, n